In [92]:
import pandas as pd
import gmaps
import numpy as np
import requests
import time
from ipywidgets.embed import embed_minimal_html

In [12]:
url = 'http://ucdpapi.pcr.uu.se/api/gedevents/19.1?pagesize=1&page=1'

filepath = 'Data_Files/ged191.xlsx'

data = pd.read_excel(filepath)

In [50]:
data_df = pd.DataFrame(data)
data_df.head()

heatmap_df = data_df[['year','country','latitude', 'longitude', 'best']]
heatmap_df


,year,country,latitude,longitude,best
0,2013,Mali,14.943290,-3.894740,31
1,2004,DR Congo (Zaire),1.769722,30.775833,200
2,2007,DR Congo (Zaire),-1.651944,28.876944,4
3,2008,India,34.378934,74.725490,2
4,2008,India,34.372601,74.167293,12
5,2011,Turkey,38.382704,40.084215,13
6,1997,India,26.000000,93.000000,1
7,2013,DR Congo (Zaire),-1.250000,29.080000,80
8,2010,Somalia,2.066667,45.366667,11
9,2003,Chad,21.358333,17.000550,2


In [54]:
# Store 'Lat' and 'Lng' into  locations 
heatmap_df = heatmap_df.loc[heatmap_df['country'] == 'Afghanistan']
#df.loc[df['column_name'] == some_value]
locations = heatmap_df[['latitude', 'longitude']].astype(float)

# Convert best death guess to float and store
deaths = heatmap_df['best'].astype(float)

years = heatmap_df['year'].astype(int).unique()
#years = years.sort()

years.astype(int)
years.sort()
years

array([1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999,
       2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [66]:
# Create figure

fig = gmaps.figure(zoom_level = 10)

heat_layer = gmaps.heatmap_layer(locations, weights=deaths, 
                             dissipating=False, max_intensity=100,
                             point_radius = .05)

fig.add_layer(heat_layer)
# fig.savefig("images/best_deaths_heatmap.png")
fig
    


ValueError: Either both zoom_level and center should be specified, or neither

In [73]:
# Create figure
for year in years:
    
    year_df = heatmap_df.loc[heatmap_df['year'] == year]
    year_locations = year_df[['latitude', 'longitude']]
    year_deaths = year_df['best']
    mean_lat = heatmap_df['latitude'].mean()
    mean_long = heatmap_df['longitude'].mean()
    mean_location = [[mean_lat, mean_long]]
    fig = gmaps.figure()

    heat_layer = gmaps.heatmap_layer(year_locations, weights=year_deaths, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = .05)

    fig.add_layer(heat_layer)
    # fig.savefig("images/best_deaths_heatmap.png")

fig

Figure(layout=FigureLayout(height='420px'))

In [116]:
afg_df = heatmap_df.loc[heatmap_df['country'] == 'Afghanistan']
mean_lat = afg_df['latitude'].mean()
mean_long = afg_df['longitude'].mean()
mean_location = [mean_lat, mean_long]

year = 2005

year_df = heatmap_df.loc[heatmap_df['year'] == year]
year_locations = year_df[['latitude', 'longitude']]
year_deaths = year_df['best']

fig = gmaps.figure(zoom_level = 5.5, center = mean_location, layout={
        'width': '500px',
        'height': '600px',
        'padding': '3px',
        'border': '1px solid black'
})

heat_layer = gmaps.heatmap_layer(year_locations, weights=year_deaths, 
                             dissipating=False, max_intensity=100,
                             point_radius = .05)

fig.add_layer(heat_layer)
# fig.savefig("images/best_deaths_heatmap.png")
#embed_minimal_html(f'afg_{year}.html', views=[fig])
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='3px', width='500px'))